# Baseline For Testing

In [ ]:
# TO RUN THIS NOTEBOOK, YOU NEED TO HAVE THE ipykernel PACKAGE INSTALLED
# YOU CAN INSTALL IT BY RUNNING `pip install ipykernel`
# OR BY UNCOMMENTING THE LINE BELOW AND RUNNING THE CELL
# You also need to have the requirements.txt installed
# !pip install ipykernel
# !pip install -r requirements.txt

In [42]:
from operator import itemgetter
import pandas as pd
from rouge_score import rouge_scorer

from typing import List, Tuple, Dict

import torch
from tqdm import tqdm

from datasets.dataset_dict import DatasetDict
from datasets import Dataset


from transformers import (
    AutoTokenizer,
    PreTrainedTokenizer,
    PreTrainedTokenizerFast,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)


# import torch.nn as nn

# import numpy as np

In [2]:
TRAIN_PATH = "../data/train.csv"
VALIDATION_PATH = "../data/validation.csv"
TEST_PATH = "../data/test_text.csv"

In [3]:
# Import the data

train_df = pd.read_csv(TRAIN_PATH)
validation_df = pd.read_csv(VALIDATION_PATH)

# The Test data corresponds to the file for submission on the Kaggle Dataset for which the labels are not available
test_df = pd.read_csv(TEST_PATH)

In [4]:
print("Train data shape: ", train_df.shape)
print(train_df.head())

print("\n")

print("Validation data shape: ", validation_df.shape)
print(validation_df.head())

Train data shape:  (21401, 2)
                                                text  \
0  Thierry Mariani sur la liste du Rassemblement ...   
1  C'est désormais officiel : Alain Juppé n'est p...   
2  La mesure est décriée par les avocats et les m...   
3  Dans une interview accordée au Figaro mercredi...   
4  Le préjudice est estimé à 2 millions d'euros. ...   

                                              titles  
0  L'information n'a pas été confirmée par l'inté...  
1  Le maire de Bordeaux ne fait plus partie des R...  
2  En 2020, les tribunaux d'instance fusionnent a...  
3  Les médecins jugés "gros prescripteurs d'arrêt...  
4  Il aura fallu mobiliser 90 gendarmes pour cett...  


Validation data shape:  (1500, 2)
                                                text  \
0  Sur les réseaux sociaux, les images sont impre...   
1  La vidéo est devenue virale. Elle montre un po...   
2  Depuis la présidentielle, il est parfois un pe...   
3  Routes endommagées, trains toujours pert

## Baseline Functions

The future functions for other implementations should keep the same input/output format for ease of use

In [5]:
# Baseline From the dataset


# Function that generates summaries using LEAD-N
def lead_summary(text: pd.Series) -> List[Tuple[int, str]]:
    """Generate summaries using the LEAD-N method

    ## Input :
    - text : pd.Series : The text data for which the summaries are to be generated (the text column from the dataset)

    ## Output :
    - summaries : List[Tuple[int, str]] : A list of Tuples containing the index of the text and the summary generated using the LEAD-N method
    """
    summaries = []
    for idx, row in text.items():
        sentences = row.split(".")
        summaries.append((idx, sentences[0] + "."))
    return summaries


# Function that generates summaries using EXT-ORACLE
def ext_oracle_summary(
    text: pd.Series,
    titles: pd.Series,
    scorer: rouge_scorer.RougeScorer,
) -> List[Tuple[int, str]]:
    """Generate summaries using the EXT-ORACLE method

    ## Input :
    - text : pd.Series : The text data for which the summaries are to be generated (the text column from the dataset)
    - titles : pd.Series : The titles of the text data (the titles column from the dataset)
    - scorer : rouge_scorer.RougeScorer : The Rouge Scorer object

    ## Output :
    - summaries : List[Tuple[int, str]] : A list of Tuples containing the index of the text and the summary generated using the EXT-ORACLE method
    """
    summaries = []
    for idx, row in text.items():
        sentences = row.split(".")
        reference = titles.iloc[idx]  # type: ignore
        rs = [scorer.score(sentence, reference)["rougeL"][2] for sentence in sentences]
        index, _ = max(enumerate(rs), key=itemgetter(1))
        summaries.append((idx, sentences[index]))
    return summaries

In [32]:
# Test the functions on the validation data

lead_summary_validation = lead_summary(validation_df["text"])

In [37]:
for idx, summary in lead_summary_validation[:5]:
    print("Lead Summary: ", summary)
    print("Reference Summary: ", validation_df["titles"].iloc[idx])  # type: ignore
    print("\n")

Lead Summary:  Sur les réseaux sociaux, les images sont impressionnantes.
Reference Summary:  Le bateau de croisière, long de 275 m, a percuté un quai lors de son arrivée dans le port de Venise, dimanche 2 juin. Quatre personnes ont été blessées.


Lead Summary:  La vidéo est devenue virale.
Reference Summary:  Le parquet de Paris a annoncé vendredi avoir ouvert une enquête après la diffusion d'une vidéo dans laquelle un policier semble tirer quasi à bout portant sur des manifestants avec un LBD lors d'échauffourées jeudi pendant le défilé contre la réforme des retraites.


Lead Summary:  Depuis la présidentielle, il est parfois un peu gêné de cette notoriété qui ne se traduira pas forcément dans les urnes.
Reference Summary:  À Trappes (Yvelines), c'est désormais la star. Benoît Hamon, nouvelle idole, et surtout des enfants.


Lead Summary:  Routes endommagées, trains toujours perturbés, agriculteurs désemparés : la Côte d'Azur tente en ce début de semaine de se remettre des dégâts ca

In [25]:
ext_oracle_summary_validation = ext_oracle_summary(
    validation_df["text"],
    validation_df["titles"],
    rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True),
)

In [26]:
for idx, summary in ext_oracle_summary_validation[:5]:
    print("EXT-ORACLE Summary: ", summary)
    print("Reference Summary: ", validation_df["titles"].iloc[idx])  # type: ignore
    print("\n")

EXT-ORACLE Summary:   Dimanche matin à Venise, l'équipage du MSC Opéra a perdu le contrôle du paquebot, à son arrivée dans le port de la cité des Doges
Reference Summary:  Le bateau de croisière, long de 275 m, a percuté un quai lors de son arrivée dans le port de Venise, dimanche 2 juin. Quatre personnes ont été blessées.


EXT-ORACLE Summary:   Elle montre un policier semblant tirer quasiment à bout portant sur des manifestations avec un lanceur de balles de défense (LBD) lors d'échauffourées survenues jeudi, pendant le défilé contre la réforme des retraites
Reference Summary:  Le parquet de Paris a annoncé vendredi avoir ouvert une enquête après la diffusion d'une vidéo dans laquelle un policier semble tirer quasi à bout portant sur des manifestants avec un LBD lors d'échauffourées jeudi pendant le défilé contre la réforme des retraites.


EXT-ORACLE Summary:   Mais à Trappes, ce n'est pas Emmanuel Macron la cible principale, mais bien Benoît Hamon
Reference Summary:  À Trappes (Yve

In [28]:
def average_rouge_score(
    summaries: List[Tuple[int, str]],
    titles: pd.Series,
    scorer: rouge_scorer.RougeScorer,
):
    """Calculate the average rouge score for the summaries generated

    ## Input :
    - summaries : [(int, str)...] : A list of Tuples containing the index of the text and the summary generated
    - text : pd.Series : The text data for which the summaries are to be generated (the text column from the dataset)
    - scorer : rouge_scorer.RougeScorer : The Rouge Scorer object

    ## Output :
    - average_rouge : float : The average rouge score for the summaries generated
    """
    rouge_scores = []
    for idx, summary in summaries:
        reference = titles.iloc[idx]  # type: ignore
        rouge_scores.append(scorer.score(summary, reference)["rougeL"][2])
    return sum(rouge_scores) / len(rouge_scores)

## New Approach

### T5

In [5]:
model_checkpoint_list = ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]
model_checkpoint = model_checkpoint_list[0]

In [6]:
# This shoud load the T5TokenizerFast from the transformers library
t5_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# t5_tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint)

# otherwise we can use thisone and compare the results
# t5_tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

In [9]:
print(t5_tokenizer)
print(type(t5_tokenizer))

T5TokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_i

In [7]:
# This should load the T5ForConditionalGeneration model from the transformers library
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [29]:
print(model)
print(type(model))

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [8]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
training_args = Seq2SeqTrainingArguments(
    output_dir=f"../outputs//{model_name}-finetuned",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    num_train_epochs=1,
    eval_steps=2,
    save_steps=2,
    warmup_steps=1,
    # overwrite_output_dir=True,
    save_total_limit=2,
)

In [11]:
def preprocess_text(
    text: str,
    title: str,
    t5_tokenizer: PreTrainedTokenizer | PreTrainedTokenizerFast,
    prefix: str = "summarize: ",
    max_input_length: int = 1024,
    max_target_length: int = 64,
) -> Dict[str, torch.Tensor]:
    """Preprocess the text data

    ## Input :
    - text : str : The text data to be preprocessed
    - title : str : The title of the text data, The Target
    - prefix : str : The prefix to be added to the text data as T5 model can be used for translation as well
    - max_input_length : int : The maximum length of the input text
    - max_target_length : int : The maximum length of the target text
    - tokenizer : t5_tokenizer : The tokenizer object

    ## Output :
    - model_inputs : Dict[str, Union[torch.Tensor, None]] : The model inputs
    """
    inputs = t5_tokenizer(
        f"{prefix} {text}",
        max_length=max_input_length,
        padding="max_length",
        truncation=True,
    )

    targets = t5_tokenizer(
        title,
        max_length=max_target_length,
        padding="max_length",
        truncation=True,
    )

    model_inputs = {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": targets.input_ids,
    }

    return model_inputs

In [15]:
def preprocess_from_df(df: pd.DataFrame):
    dataframe_list = []
    for i in range(len(df)):
        dataframe_list.append(
            preprocess_text(df["text"].iloc[i], df["titles"].iloc[i], t5_tokenizer)
        )

    return Dataset.from_list(dataframe_list)

In [20]:
# Construct metric
rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = t5_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge_scores = [
        rouge.score(pred, label)["rougeL"].fmeasure
        for pred, label in zip(decoded_preds, decoded_labels)
    ]

    return {"rougeL_fmeasure": sum(rouge_scores) / len(rouge_scores)}

### Running

In [13]:
# Construct the DataCollector
data_collator = DataCollatorForSeq2Seq(t5_tokenizer, model=model)

In [16]:
train_dataset = preprocess_from_df(train_df)
validation_dataset = preprocess_from_df(validation_df)

In [18]:
total_dataset = DatasetDict({"train": train_dataset, "validation": validation_dataset})
total_dataset

In [21]:
# Construct the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=total_dataset["train"],
    eval_dataset=total_dataset["validation"],
)

/home/shaamallow/.miniforge3/envs/datachallenge/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
trainer.train()

 19%|█▊        | 500/2676 [06:59<24:13,  1.50it/s]  

{'loss': 2.159, 'grad_norm': 1.5339239835739136, 'learning_rate': 4.067289719626168e-05, 'epoch': 0.19}


 37%|███▋      | 1000/2676 [13:28<18:08,  1.54it/s]

{'loss': 1.973, 'grad_norm': 1.4470257759094238, 'learning_rate': 3.1327102803738314e-05, 'epoch': 0.37}


 56%|█████▌    | 1500/2676 [20:01<12:57,  1.51it/s]

...

 93%|█████████▎| 2500/2676 [35:48<05:24,  1.85s/it]

{'loss': 1.9142, 'grad_norm': 1.627769947052002, 'learning_rate': 3.2897196261682244e-06, 'epoch': 0.93}


100%|█████████▉| 2675/2676 [42:07<00:03,  3.67s/it]/home/shaamallow/.miniforge3/envs/datachallenge/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                   
100%|██████████| 2676/2676 [42:53<00:00,  1.04it/s]

{'eval_loss': 1.7585475444793701, 'eval_rougeL_fmeasure': 0.166479094478431, 'eval_runtime': 44.9549, 'eval_samples_per_second': 33.367, 'eval_steps_per_second': 4.182, 'epoch': 1.0}
{'train_runtime': 2573.4343, 'train_samples_per_second': 8.316, 'train_steps_per_second': 1.04, 'train_loss': 1.9709011129199656, 'epoch': 1.0}


TrainOutput(global_step=2676, training_loss=1.9709011129199656, metrics={'train_runtime': 2573.4343, 'train_samples_per_second': 8.316, 'train_steps_per_second': 1.04, 'train_loss': 1.9709011129199656, 'epoch': 1.0})

### Evaluation

In [43]:
def t5_summary(
    text: pd.Series, t5_tokenizer: PreTrainedTokenizer, model: AutoModelForSeq2SeqLM
):
    """Generate summaries using the T5 model

    Using the standard representation defined in baseline part of notebook

    ## Input :
    - text : pd.Series : The text data for which the summaries are to be generated (the text column from the dataset)
    - t5_tokenizer : PreTrainedTokenizer : The tokenizer object
    - model : AutoModelForSeq2SeqLM : The model object

    ## Output :
    - summaries : List[Tuple[int, str]] : A list of Tuples containing the index of the text and the summary generated using the T5 model
    """
    summaries = []
    for idx, row in tqdm(text.items()):
        input_text = t5_tokenizer(row, return_tensors="pt").input_ids.to(model.device)
        output = model.generate(
            input_text,
            max_length=64,
            early_stopping=True,
            num_return_sequences=1,
        )
        summaries.append(
            (idx, t5_tokenizer.decode(output[0], skip_special_tokens=True))
        )
    return summaries

In [45]:
# Run title generation for submission
t5_summary_kaggle = t5_summary(test_df["text"], t5_tokenizer, model)

0it [00:00, ?it/s]/home/shaamallow/.miniforge3/envs/datachallenge/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
1500it [05:42,  4.37it/s]


In [46]:
t5_summary_kaggle_df = pd.DataFrame(t5_summary_kaggle, columns=["ID", "titles"])
t5_summary_kaggle_df.to_csv("../outputs/submissions//t5_summary_kaggle.csv", index=False)

### Testing

In [12]:
print(preprocess_text("This is a test", "This is a test", t5_tokenizer))

{'input_ids': [21603, 10, 100, 19, 3, 9, 794, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
# Check Encoding/Decoding size embeddings
test_text = train_df["text"].iloc[0]
test_title = train_df["titles"].iloc[0]

print(test_text)
print(test_title)

preprocess_test = preprocess_text(test_text, test_title, t5_tokenizer)

print(preprocess_test.keys())

decoded_text = t5_tokenizer.decode(preprocess_test["input_ids"])
decoded_title = t5_tokenizer.decode(preprocess_test["labels"])

print(decoded_text)
print(decoded_title)

Thierry Mariani sur la liste du Rassemblement national (RN, ex-FN) aux européennes ? C'est ce qu'affirme mardi 11 septembre Chez Pol, la nouvelle newsletter politique de Libération. L'ancien député Les Républicain et ministre de Nicolas Sarkozy serait sur le point de rejoindre les troupes de Marine Le Pen pour le élections européennes de 2019. "Ça va se faire. Ce n'est plus qu'une question de calendrier. On n'est pas obligé de l'annoncer tout de suite, à huit mois des européennes", aurait ainsi assuré un membre influent du RN. Contacté par Franceinfo, M. Mariani n'a pas confirmé l'information. "Les élections sont en juin, je ne sais même pas qui sera numéro 1 sur la liste", a répondu l'ancien ministre des Transports. Il reconnaît toutefois, toujours cité par Franceinfo, que son nom sur la liste du RN "fait partie des possibilités". "Fréjus est une ville sympathique mais je n'ai pas prévu de m'y rendre ce week_end", a-t-il par ailleurs commenté sur Twitter alors que Marine Le Pen réunit

In [39]:
test_text = train_df["text"].iloc[1]
test_title = train_df["titles"].iloc[1]

input_text = t5_tokenizer(test_text, return_tensors="pt").input_ids.to(model.device)

output = model.generate(
    input_text,
    max_length=64,
    early_stopping=True,
    num_return_sequences=1,
)
print(output)
print(t5_tokenizer.decode(output[0], skip_special_tokens=True))
print(test_title)

print(test_text)

tensor([[    0,   312,   187,    60,    20, 20820,     3,     9,  1136,    29,
           106,    75,   154,    73, 26263,  1194,   285,     3,     7,    31,
            32,   102,  2970,     6,     3,  5928,   759,     6,    20,    50,
         16872,  2676,     3,    40,    31, 10398, 14523, 16872,     5,     1]],
       device='cuda:0')
Le maire de Bordeaux a dénoncé un glissement qui s'opère, selon lui, de la droite vers l'extême droite.
Le maire de Bordeaux ne fait plus partie des Républicains et il tient à le montrer. Lors de ses voeux à la presse, l'édile a pris ses distances avec sa famille politique historique, qu'il trouve trop proche de Marine Le Pen.
C'est désormais officiel : Alain Juppé n'est plus membre des Républicains. L'ex-Premier ministre de Jacques Chirac, cofondateur de l'UMP en 2002, ne paie plus sa cotisation auprès du parti de droite. Mercredi 9 janvier, le maire de Bordeaux a dénoncé un glissement qui s'opère, selon lui, de la droite vers l'extême droite. "Je m